In [1]:
#import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv('CLEAN.csv',encoding ='ISO-8859-1')

In [3]:
data.shape

(5169, 17)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5169 entries, 0 to 5168
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            5169 non-null   int64  
 1   id                    5169 non-null   int64  
 2   title                 5169 non-null   object 
 3   genres                5169 non-null   object 
 4   popularity            5169 non-null   float64
 5   production_companies  5169 non-null   object 
 6   release_date          5169 non-null   object 
 7   budget                5169 non-null   float64
 8   revenue               5169 non-null   float64
 9   runtime               5169 non-null   float64
 10  vote_average          5169 non-null   float64
 11  vote_count            5169 non-null   float64
 12  month                 5169 non-null   int64  
 13  year                  5169 non-null   int64  
 14  day_of_week           5169 non-null   int64  
 15  weighted_vote        

In [5]:
data['release_date'] = pd.to_datetime(data['release_date'])

In [6]:
data.head(3)

,Unnamed: 0,id,title,genres,popularity,production_companies,release_date,budget,revenue,runtime,vote_average,vote_count,month,year,day_of_week,weighted_vote,popularity_category
0,0,76600,Avatar: The Way of Water,Science Fiction,9366.788,20th Century Studios,2022-12-14,350000000.0,2.312336e+09,192.0,7.751,6748.0,12,2022,2,7.589539,High
1,1,505642,Black Panther: Wakanda Forever,Action,2525.408,Marvel Studios,2022-11-09,250000000.0,8.585356e+08,162.0,7.338,3922.0,11,2022,2,7.167744,High
2,2,315162,Puss in Boots: The Last Wish,Animation,2078.280,DreamWorks Animation,2022-12-07,90000000.0,4.630876e+08,103.0,8.363,4671.0,12,2022,2,8.034487,High


In [7]:
data.isna().sum()


Unnamed: 0              0
id                      0
title                   0
genres                  0
popularity              0
production_companies    0
release_date            0
budget                  0
revenue                 0
runtime                 0
vote_average            0
vote_count              0
month                   0
year                    0
day_of_week             0
weighted_vote           0
popularity_category     0
dtype: int64

In [8]:
df=data.copy()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5169 entries, 0 to 5168
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unnamed: 0            5169 non-null   int64         
 1   id                    5169 non-null   int64         
 2   title                 5169 non-null   object        
 3   genres                5169 non-null   object        
 4   popularity            5169 non-null   float64       
 5   production_companies  5169 non-null   object        
 6   release_date          5169 non-null   datetime64[ns]
 7   budget                5169 non-null   float64       
 8   revenue               5169 non-null   float64       
 9   runtime               5169 non-null   float64       
 10  vote_average          5169 non-null   float64       
 11  vote_count            5169 non-null   float64       
 12  month                 5169 non-null   int64         
 13  year              

In [10]:
df = df.drop(['Unnamed: 0','id', 'title','popularity', 'release_date','vote_average','vote_count','weighted_vote','popularity_category'], axis=1)

### Scaling

In [11]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale

#### Scaling of data with outliers

In [13]:
from sklearn.preprocessing import MinMaxScaler

# Create a copy of the original DataFrame
scaled_df = df.copy()

# Select the continuous numerical columns for scaling, here avoid revenue column since it is the target column
#columns_to_scale = [ 'budget','runtime']  # Replace with the actual column names

# Initialize the MinMaxScaler
scaler1 = MinMaxScaler()
scaler2=MinMaxScaler()

# Scale the selected columns in the scaled DataFrame
scaled_df['budget'] = scaler1.fit_transform(df[['budget']])
scaled_df['runtime'] = scaler2.fit_transform(df[['runtime']])
# Display the scaled DataFrame
scaled_df

,genres,production_companies,budget,revenue,runtime,month,year,day_of_week
0,Science Fiction,20th Century Studios,0.699940,2.312336e+09,0.565089,12,2022,2
1,Action,Marvel Studios,0.499900,8.585356e+08,0.476331,11,2022,2
2,Animation,DreamWorks Animation,0.179836,4.630876e+08,0.301775,12,2022,2
3,Comedy,Playtone,0.099820,1.038423e+08,0.369822,12,2022,2
4,Action,Lionsgate,0.029806,4.495365e+06,0.295858,12,2022,2
...,...,...,...,...,...,...,...,...
5164,Action,WBK Cineplexs,0.000000,2.000000e+05,0.002959,10,2021,6
5165,Documentary,Empire Productions,0.029806,1.000000e+06,0.263314,8,2020,4
5166,Horror,Ironopolis Film Company,0.029806,1.094584e+06,0.248521,7,2008,1
5167,Science Fiction,GYR Films,0.001800,1.260000e+05,0.292899,3,2019,4


## Model Building

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.metrics import accuracy_score

In [15]:
x=scaled_df.drop(['revenue'],axis=1)
y=scaled_df[['revenue']]

In [16]:
from sklearn.preprocessing import LabelEncoder
le1=LabelEncoder()
x[['genres']]=x[['genres']].apply(le1.fit_transform)
le2=LabelEncoder()
x[['production_companies']]=x[['production_companies']].apply(le2.fit_transform)


In [17]:
x.head()

,genres,production_companies,budget,runtime,month,year,day_of_week
0,14,7,0.699940,0.565089,12,2022,2
1,0,913,0.499900,0.476331,11,2022,2
2,2,442,0.179836,0.301775,12,2022,2
3,3,1141,0.099820,0.369822,12,2022,2
4,0,840,0.029806,0.295858,12,2022,2


In [18]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [19]:
from catboost import CatBoostRegressor 
from sklearn.metrics import r2_score
cat = CatBoostRegressor()
mod=cat.fit(x_train, y_train)
# Make predictions on the test set
y_pred = mod.predict(x_test)

# Calculate the R^2 score
r2 = r2_score(y_test, y_pred)

print("R^2 score:", r2)



Learning rate set to 0.051237
0:	learn: 186770923.4688044	total: 157ms	remaining: 2m 36s
1:	learn: 182379608.7080829	total: 186ms	remaining: 1m 32s
2:	learn: 177861543.0604298	total: 193ms	remaining: 1m 4s
3:	learn: 173723029.1069593	total: 199ms	remaining: 49.5s
4:	learn: 169602868.7034629	total: 203ms	remaining: 40.3s
5:	learn: 165927241.8857062	total: 206ms	remaining: 34.1s
6:	learn: 162747568.3526656	total: 210ms	remaining: 29.8s
7:	learn: 159546905.8870580	total: 213ms	remaining: 26.4s
8:	learn: 156486513.3424606	total: 216ms	remaining: 23.8s
9:	learn: 153753390.4232129	total: 219ms	remaining: 21.7s
10:	learn: 150931877.5778810	total: 223ms	remaining: 20s
11:	learn: 148270087.3612796	total: 226ms	remaining: 18.6s
12:	learn: 145994508.8149019	total: 231ms	remaining: 17.5s
13:	learn: 143985413.6160119	total: 236ms	remaining: 16.6s
14:	learn: 141878500.3564115	total: 242ms	remaining: 15.9s
15:	learn: 139933886.8405084	total: 246ms	remaining: 15.2s
16:	learn: 138033444.2238348	total: 

In [20]:
CBR_pkl=open('CBR.pkl','wb')
pickle.dump(mod,CBR_pkl)
CBR_pkl.close()

In [21]:
#Label Encoder Pickle 
leR1_pkl = open('leR1.pkl','wb')
pickle.dump(le1,leR1_pkl)
leR1_pkl.close()

In [22]:
#Label Encoder Pickle 
leR2_pkl = open('leR2.pkl','wb')
pickle.dump(le2,leR2_pkl)
leR2_pkl.close()

In [23]:
SC1_pkl = open('SC1.pkl','wb')
pickle.dump(scaler1,SC1_pkl)
SC1_pkl.close()

In [24]:
SC2_pkl = open('SC2.pkl','wb')
pickle.dump(scaler2,SC2_pkl)
SC2_pkl.close()

In [25]:
le_1=pickle.load(open('leR1.pkl','rb'))

In [26]:
le_2=pickle.load(open('leR2.pkl','rb'))

In [27]:
sc_1=pickle.load(open('SC1.pkl','rb'))

In [28]:
sc_2=pickle.load(open('SC2.pkl','rb'))